In [1]:
# -*- coding:utf-8 -*-
#处理测试集

import codecs
import collections
from collections import Iterable
import re
import torch
def originHandle():
    with open('./renmin.txt', 'r',encoding='utf8') as inp, open('./renmin2.txt', 'w') as outp:
        for line in inp.readlines():
            line = line.split('  ')
            i = 1
            while i < len(line) - 1:
                if line[i][0] == '[':
                    outp.write(line[i].split('/')[0][1:])
                    i += 1
                    while i < len(line) - 1 and line[i].find(']') == -1:
                        if line[i] != '':
                            outp.write(line[i].split('/')[0])
                        i += 1
                    outp.write(line[i].split('/')[0].strip() + '/' + line[i][-2:]+' ')
                elif line[i].split('/')[1] == 'nr':
                    word = line[i].split('/')[0]
                    i += 1
                    if i < len(line) - 1 and line[i].split('/')[1] == 'nr':
                        outp.write(word + line[i].split('/')[0] + '/nr ')
                    else:
                        outp.write(word + '/nr ')
                        continue
                else:
                    outp.write(line[i] + ' ')
                i += 1
            outp.write('\n')
def originHandle2():
    with codecs.open('renmin2.txt', 'r', encoding='gbk') as inp, codecs.open('renmin3.txt', 'w', encoding='gbk') as outp:
        for line in inp.readlines():
            line = line.split(' ')
            i = 0
            while i<len(line)-1:
                if line[i]=='':
                    i+=1
                    continue
                word = line[i].split('/')[0]
                tag = line[i].split('/')[1]
                if tag=='nr' or tag=='ns' or tag=='nt':
                    outp.write(word[0]+"/B ")
                    for j in word[1:len(word)]:
                        if j!=' ':
                            outp.write(j+"/I ")
                else:
                    for wor in word:
                        outp.write(wor+'/O ')
                i+=1
            outp.write('\n')
def sentence2split():
    with codecs.open('./renmin3.txt','r','gbk') as inp,codecs.open('./renmin4.txt','w','gbk') as outp:
        texts = inp.read()
        sentences = re.split('[，。！？、‘’“”:]/[O]', texts)
        for sentence in sentences:
	        if sentence != " ":
		        outp.write(sentence.strip()+'\n')
def originHandle3():
    with open('./validation.txt', 'r',encoding='utf8') as inp, open('./validation2.txt', 'w') as outp:
        for line in inp.readlines():
            line = line.split('  ')
            i = 1
            while i < len(line) - 1:
                if line[i][0] == '[':
                    outp.write(line[i].split('/')[0][1:])
                    i += 1
                    while i < len(line) - 1 and line[i].find(']') == -1:
                        if line[i] != '':
                            outp.write(line[i].split('/')[0])
                        i += 1
                    outp.write(line[i].split('/')[0].strip() + '/' + line[i][-2:]+' ')
                elif line[i].split('/')[1] == 'nr':
                    word = line[i].split('/')[0]
                    i += 1
                    if i < len(line) - 1 and line[i].split('/')[1] == 'nr':
                        outp.write(word + line[i].split('/')[0] + '/nr ')
                    else:
                        outp.write(word + '/nr ')
                        continue
                else:
                    outp.write(line[i] + ' ')
                i += 1
            outp.write('\n')
def originHandle4():
    with codecs.open('validation2.txt', 'r', encoding='gbk') as inp, codecs.open('validation3.txt', 'w', encoding='gbk') as outp:
        for line in inp.readlines():
            line = line.split(' ')
            i = 0
            while i<len(line)-1:
                if line[i]=='':
                    i+=1
                    continue
                word = line[i].split('/')[0]
                tag = line[i].split('/')[1]
                if tag=='nr' or tag=='ns' or tag=='nt':
                    outp.write(word[0]+"/B ")
                    for j in word[1:len(word)]:
                        if j!=' ':
                            outp.write(j+"/I ")
                else:
                    for wor in word:
                        outp.write(wor+'/O ')
                i+=1
            outp.write('\n')
def sentence2split1():
    with codecs.open('./validation3.txt','r','gbk') as inp,codecs.open('./validation4.txt','w','gbk') as outp:
        texts = inp.read()
        sentences = re.split('[，。！？、‘’“”:]/[O]', texts)
        for sentence in sentences:
	        if sentence != " ":
		        outp.write(sentence.strip()+'\n')
originHandle()
originHandle2()
sentence2split()
originHandle3()
originHandle4()
sentence2split1()
print("Done")

<ipython-input-1-a2c67b6213a4>:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable


Done


In [1]:

import codecs

datas = list()
labels = list()

datas1 = list()
labels1 = list()

#打开训练集，处理x，y
with codecs.open('renmin4.txt','r','gbk') as train:
    for line in train.readlines():
        line = line.split()
        linedata = []
        linelabel = []
        numNotO = 0
        for word in line:
            word = word.split('/')

            linedata.append(word[0])
            linelabel.append(word[1])
            if word[1] != 'O':
                numNotO += 1
        if numNotO != 0:
            datas.append(linedata)
            labels.append(linelabel)
#打开验证集，处理x，y
with codecs.open('validation4.txt','r','gbk') as val:
    for line in val.readlines():
        line = line.split()
        linedata1 = []
        linelabel1 = []
        numNotO1 = 0
        for word in line:
            word = word.split('/')
            linedata1.append(word[0])
            linelabel1.append(word[1])
            if word[1] != 'O':
                numNotO1 += 1
        if numNotO1 != 0:
            datas1.append(linedata1)
            labels1.append(linelabel1)

print('数据读取成功')



数据读取成功


In [2]:
#调用训练好的词向量
import codecs
import numpy as np
from annoy import AnnoyIndex
class PreTrainedEmbeddings(object):
    """ A wrapper around pre-trained word vectors and their use """
    def __init__(self, word_to_index, word_vectors):
        """
        Args:
            word_to_index (dict): mapping from word to integers
            word_vectors (list of numpy arrays)
        """
        self.word_to_index = word_to_index
        self.word_vectors = word_vectors
        self.index_to_word = {v: k for k, v in self.word_to_index.items()}

        self.index = AnnoyIndex(len(word_vectors[0]), metric='euclidean')
        print("Building Index!")
        for _, i in self.word_to_index.items():
            self.index.add_item(i, self.word_vectors[i])
        self.index.build(50)
        print("Finished!")

    @classmethod
    def from_embeddings_file(cls, embedding_file):
        """Instantiate from pre-trained vector file.

        Vector file should be of the format:
            word0 x0_0 x0_1 x0_2 x0_3 ... x0_N
            word1 x1_0 x1_1 x1_2 x1_3 ... x1_N

        Args:
            embedding_file (str): location of the file
        Returns:
            instance of PretrainedEmbeddigns
        """
        word_to_index = {}
        word_vectors = []

        with codecs.open(embedding_file,encoding='utf-8') as fp:
            for line in fp.readlines():
                line = line.split(" ")
                word = line[0]
                line = [x.strip() for x in line if x.strip()!='']
                vec = np.array([float(x) for x in line[1:]])

                word_to_index[word] = len(word_to_index)
                word_vectors.append(vec)

        return cls(word_to_index, word_vectors)

    def get_embedding(self, word):
        """
        Args:
            word (str)
        Returns
            an embedding (numpy.ndarray)
        """
        return self.word_vectors[self.word_to_index[word]]

embeddings = PreTrainedEmbeddings.from_embeddings_file('ctb.50d.txt')




Building Index!
Finished!


In [3]:
import torch.optim as optim
import torch.nn as nn
import torch
id2tag={0:'O',1:'B',2:'I'}
def calculate(i,x,y,res=[]):
    entity=[]
    for j in range(len(x)):
        if int(y[j][0])==0:
            continue
        if id2tag[int(y[j][0])]=='B':
            entity=[datas1[i][j]]
        elif id2tag[int(y[j][0])]=='I'  and len(entity)!=0 and j!=len(x)-1:
            if id2tag[int(y[j+1][0])]!='O':
                entity.append(datas1[i][j])
            else:
                entity.append(datas1[i][j])
                res.append(entity)
                entity=[]
        elif id2tag[int(y[j][0])]=='I'  and len(entity)!=0 and j==len(x)-1:
            entity.append(datas1[i][j])
            res.append(entity)
            entity=[]
        else:
            entity=[]
    return res
def calculate1(i,x,y,res=[]):
    entity=[]
    for j in range(len(x)):
        if y[j]==0:
            continue
        if id2tag[y[j]]=='B':
            entity=[datas1[i][j]]
        elif id2tag[y[j]]=='I'  and len(entity)!=0 and j!=len(x)-1:
            if id2tag[y[j+1]]!='O':
                entity.append(datas1[i][j])
            else:
                entity.append(datas1[i][j])
                res.append(entity)
                entity=[]
        elif id2tag[y[j]]=='I'  and len(entity)!=0 and j==len(x)-1:
            entity.append(datas1[i][j])
            res.append(entity)
            entity=[]
        else:
            entity=[]
    return res


In [4]:
#将训练集的x变成词向量
x_train=[]
for s1 in datas:
    line=[]
    for s2 in s1:
        try:
            emb=embeddings.get_embedding(s2)
            line.append(emb)
        except KeyError:
            emb=embeddings.get_embedding('-unknown-')
            line.append(emb)
    x_train.append(line)


y_train=labels
for l1 in range(len(y_train)):
    for l2 in range(len(y_train[l1])):
        if y_train[l1][l2]=='O':
            y_train[l1][l2]=0
        elif y_train[l1][l2]=='B':
            y_train[l1][l2]=1
        else:
            y_train[l1][l2]=2
#将验证集的x变成词嵌入编码
x_val=[]
for s11 in datas1:
    line1=[]
    for s22 in s11:
        try:
            emb1=embeddings.get_embedding(s22)
            line1.append(emb1)
        except KeyError:
            emb1=embeddings.get_embedding('-unknown-')
            line1.append(emb1)
    x_val.append(line1)
y_val=labels1
for l1 in range(len(y_val)):
    for l2 in range(len(y_val[l1])):
        if y_val[l1][l2]=='O':
            y_val[l1][l2]=0
        elif y_val[l1][l2]=='B':
            y_val[l1][l2]=1
        else:
            y_val[l1][l2]=2

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
class BiLSTM(nn.Module):
    def __init__(self,input_size,hidden_size,out_size):
        super(BiLSTM, self).__init__()
        self.hidden_size=hidden_size
        self.out_size=out_size
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, out_size)

    def forward(self, X):
        # X: [batch_size, max_len, n_class]
        batch_size = X.shape[0]
        input = X.transpose(0,1)
        hidden_state = torch.randn(1*2, batch_size, self.hidden_size)   #
        cell_state = torch.randn(1*2, batch_size, self.hidden_size)     #定义初始参数

        outputs, (_, _) = self.lstm(input, (hidden_state, cell_state))
        model = self.fc(outputs)
        model=torch.squeeze(model)
        return torch.softmax(model,dim=1)

In [15]:
def predict(x):
        # Apply softmax to output
        output = model(x)
        pred = output.data.max(1, keepdim=True)[1]
        return pred
model=BiLSTM(50,50,3)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion=nn.CrossEntropyLoss()
num_epoch = 30# 最大迭代更新次数
for epoch in range(num_epoch):
    i1=0
    losses=0
    for ar in x_train:
        arr=[]
        arr.append(ar)
        arr=torch.FloatTensor(arr)
        optimizer.zero_grad()
        y_p = model(arr)
        trainy=torch.LongTensor(y_train[i1])
        loss = criterion(y_p,trainy)# 计算损失
        losses+=loss
        loss.backward()  # 计算梯度，误差回传
        optimizer.step()#根据计算的梯度，更新网络中的参数
        i1+=1
    if epoch % 1 == 0:
        print('epoch: {}, loss: {}'.format(epoch, losses/i1))
        entityres=[]
        entityall=[]
        i2=0
        f1=0
        for ar1 in x_val:
            arr1=[]
            arr1.append(ar1)
            arr1=torch.FloatTensor(arr1)
            y_p1 = predict(arr1)
            entityres = calculate(i2,ar1,y_p1,entityres)
            entityall = calculate1(i2,ar1,y_val[i2],entityall)
            i2+=1
        jiaoji = [i for i in entityres if i in entityall]
        if len(jiaoji)!=0:
            zhun = float(len(jiaoji))/len(entityres)
            quan = float(len(jiaoji))/len(entityall)
            f1=(2*zhun*quan)/(zhun+quan)
            print ("val:")
            print ("查准率:", zhun)
            print ("查全率:", quan)
            print ("f1:", f1)
            print("\n")
        else:
            print ("查准率:",0)
            print("\n")
        if f1>0.76:
            break
print("Done")
torch.save(model.state_dict(),"work23.pt")


KeyboardInterrupt: 

In [24]:
def predict(x):
        # Apply softmax to output
        output = model1(x)
        pred = output.data.max(1, keepdim=True)[1]
        return pred
model1=BiLSTM(50,50,3)
model1.load_state_dict(torch.load("work23.pt"))
s="北 京 理 工 大 学"
s=s.split()
s1=[]
i=0
for l in s:
    s[i]=embeddings.get_embedding(l)
    i+=1
s1.append(s)
s1=torch.FloatTensor(s1)
ss=predict(s1)
print(ss)


tensor([[1],
        [2],
        [2],
        [2],
        [2],
        [2]])
